In [ ]:
% matplotlib inline
import MySQLdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
db = MySQLdb.connect(host="crunchbase.ckfkhzm7cot3.us-west-2.rds.amazonaws.com",
                     port=3306,
                     user="root",         # your username
                     passwd="root1234",  # your password
                     db="analytics_2")        # name of the data base


In [ ]:
cur = db.cursor()

In [455]:
len(persons)

NameError: name 'persons' is not defined

In [ ]:
count = cur.execute("SELECT count(*) FROM cb_objects")

#### Load into pandas dataframe from database

In [ ]:
df_mysql = pd.read_sql('select * from cb_objects LIMIT 100;', con=db)

In [456]:
persons = pd.read_sql('Select * from cb_objects where entity_type = \'Person\'', con=db)

In [ ]:
relationships = pd.read_sql('Select * from cb_relationships', con=db)

In [ ]:
degrees = pd.read_sql('Select * from cb_degrees', con=db)

In [476]:
funding_rounds = pd.read_sql('Select * from cb_funding_rounds', con=db)

In [421]:
baseline_data = pd.read_sql("Select * from cb_master_data_2", con=db)

In [ ]:
companies = pd.read_sql('Select * from cb_objects where entity_type = \'Company\'', con=db)

In [ ]:
degrees = baseline_data

In [ ]:
baseline_data[:5]

#### Fill nan for empty and None values

In [ ]:
degrees.fillna(value=np.nan, inplace=True)

In [ ]:
persons.fillna(value=np.nan, inplace=True)
relationships.fillna(value=np.nan, inplace=True)
degrees.fillna(value=np.nan, inplace=True)

#### People

For the baseline model we need to consider only people who are Founders, CEO, CIO etc

#### Work Experience

In [457]:
len(persons)

226708

In [459]:
work_ex = pd.merge(persons, relationships, how='inner', right_on='person_object_id', left_on='id')

In [460]:
print 'Total number of work ex rows', len(work_ex)

Total number of work ex rows 401957


In [461]:
workex_with_no_persons = work_ex[work_ex['id_x'].isnull()]
person_with_no_workex = work_ex[work_ex['id_y'].isnull()]
print "workex rows with no corresponding person",len(workex_with_no_persons)
print "Persons with no details about their workex",len(person_with_no_workex)

workex rows with no corresponding person 0
Persons with no details about their workex 0


In [462]:
print "Number of rows with no start_at for workex is: ",len(work_ex[work_ex['start_at'].isnull()]), "which is roughly half the number of rows"

Number of rows with no start_at for workex is:  195122 which is roughly half the number of rows


Idea : We can fill the start_at date with one of the following:
1. founding_date of the company
2. created_at_y which is when the record was created in the database

To analyse we can first build a plot of the different between the 2 and see the relation between the title, created at and founding date. It can mean that most of the founders have their profile created on crunchbase much later than the founding date, in which case we can safely use the founding date. However, for employess we an use the created_at.

In [507]:
work_ex['title'].unique

<bound method Series.unique of 0                         Co-Founder/CEO/Board of Directors
1                                           Co-founder & VP
2                                      Business Development
3                                           Limited Partner
4                                                Consultant
5                                                Consultant
6                                                Consultant
7                                                Consultant
8                                              VP Marketing
9                                           Limited Partner
10                                                         
11                                                         
12                                                         
13                                                         
14                                                         
15                                                         
16       

This below line of code filters all the work ex where the person is one of the founding members or the board of directors

In [509]:
len(work_ex)

401957

In [520]:
founders = work_ex.loc[work_ex['id_x'].isin(work_ex.loc[work_ex['title'].str.contains('Cofounder', regex=True),'id_x'])]

In [521]:
founders['title']

1625                                          Board
1626                                      Cofounder
3454      CoFounder & Vice President of Engineering
3455                                Cofounder & CEO
3456                                Product Manager
3457                                            CEO
3458                                Cofounder & CEO
3459                                        Advisor
3460                                        Advisor
3461                                        Advisor
4270                                     NY Partner
4271                                   Board Member
4272                                      Cofounder
4273                               Business Advisor
4274                               Managing Partner
4275                                            EVP
4276                        Regional Advisory Board
4277       Co-Chairman & Co-Chief Executive Officer
4278                                 Advisory Board
4279        

In [492]:
founders = work_ex.loc[work_ex['id_x'].isin(work_ex.loc[work_ex['title'].str.contains('Chief|C[A-Z]O|Co\-Founder|Co\-founder|Board', regex=True),'id_x'])]

In [465]:
founders['start_at'] = pd.to_datetime(founders['start_at'])
# import dateutil
# founderspd.to_datetime(founders['start_at'])
# founders['start_at']
# len(founders[founders['start_at'].isnull()])

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [ ]:
founders.groupby('id_x').min()#agg({'start_at':min})

In [ ]:
founder_groups = founders.groupby('id_x')#.agg({'start_at':pd.datetime})
founder_groups['start_at'].min()
founders[founders['id_x']=='p:99969']

In [468]:
import datetime
# pd.to_datetime(founders['start_at'])[5] > pd.to_datetime('2012-01-01')
np.min(pd.to_datetime(founders['start_at']))

Timestamp('1960-01-01 00:00:00')

TODO: Aggregate as the min of teh start_at, this will give the first date when work ex added

In [500]:
len(founders['relationship_object_id'].unique())

87682

In [477]:
funding_rounds

,id,funding_round_id,object_id,funded_at,funding_round_type,funding_round_code,raised_amount_usd,raised_amount,raised_currency_code,pre_money_valuation_usd,...,post_money_valuation,post_money_currency_code,participants,is_first_round,is_last_round,source_url,source_description,created_by,created_at,updated_at
0,1,1,c:4,2006-12-01,series-b,b,8500000.0,8500000.0,USD,NaN,...,NaN,None,2,0,0,http://www.marketingvox.com/archives/2006/12/2...,None,initial-importer,2007-07-04 04:52:57,2008-02-27 23:14:29
1,2,2,c:5,2004-09-01,angel,angel,500000.0,500000.0,USD,NaN,...,NaN,USD,2,0,1,None,None,initial-importer,2007-05-27 06:08:18,2013-06-28 20:07:23
2,3,3,c:5,2005-05-01,series-a,a,12700000.0,12700000.0,USD,115000000.0,...,NaN,USD,3,0,0,http://www.techcrunch.com/2007/11/02/jim-breye...,Jim Breyer: Extra $500 Million Round For Faceb...,initial-importer,2007-05-27 06:09:10,2013-06-28 20:07:23
3,4,4,c:5,2006-04-01,series-b,b,27500000.0,27500000.0,USD,525000000.0,...,NaN,USD,4,0,0,http://www.facebook.com/press/info.php?factsheet,Facebook Funding,initial-importer,2007-05-27 06:09:36,2013-06-28 20:07:24
4,5,5,c:7299,2006-05-01,series-b,b,10500000.0,10500000.0,USD,NaN,...,NaN,None,2,0,0,http://www.techcrunch.com/2006/05/14/photobuck...,PhotoBucket Closes $10.5M From Trinity Ventures,initial-importer,2007-05-29 11:05:59,2008-04-16 17:09:12
5,6,6,c:9,2007-01-01,series-a,a,1500000.0,1500000.0,USD,8500000.0,...,10000000.0,USD,1,0,1,None,None,initial-importer,2007-05-31 20:19:28,2009-01-19 18:29:39
6,7,7,c:9,2007-03-01,series-b,b,10000000.0,10000000.0,USD,100000000.0,...,110000000.0,USD,1,0,0,http://www.techcrunch.com/2007/03/05/100-milli...,100 Million Valuation for Geni,initial-importer,2007-05-31 20:35:03,2009-01-19 18:29:39
7,8,8,c:13,2005-12-01,series-a,seed,1500000.0,1500000.0,USD,NaN,...,NaN,None,4,0,0,http://www.techcrunch.com/2006/11/14/stumbleup...,StumbleUpon May Be For Sale: 50M,initial-importer,2007-06-01 19:14:34,2011-03-10 02:38:18
8,9,9,c:14,2007-05-01,series-a,a,6300000.0,6300000.0,USD,NaN,...,NaN,None,2,0,1,http://www.techcrunch.com/2007/05/31/gizmoz-la...,None,initial-importer,2007-06-01 20:09:47,2008-03-17 04:11:44
9,10,10,c:15,2006-06-01,angel,seed,12000.0,12000.0,USD,NaN,...,NaN,None,1,0,1,http://www.techcrunch.com/2007/03/06/scribd-yo...,"Scribd ""YouTube for Documents?"" Gets $300K",initial-importer,2007-06-02 07:36:21,2012-08-13 23:30:48


In [502]:
tabl = pd.merge(founders, funding_rounds,how='inner', left_on='relationship_object_id', right_on='object_id')

In [503]:
tabl[['funding_round_id','name','title']]

,funding_round_id,name,title
0,888,Ben Elowitz,Co-Founder/CEO/Board of Directors
1,889,Ben Elowitz,Co-Founder/CEO/Board of Directors
2,2312,Ben Elowitz,Co-Founder/CEO/Board of Directors
3,888,Bert Hogue,CFO
4,889,Bert Hogue,CFO
5,2312,Bert Hogue,CFO
6,888,Rob Grady,COO/Board of Directors
7,889,Rob Grady,COO/Board of Directors
8,2312,Rob Grady,COO/Board of Directors
9,888,Rob Grady,"Senior Vice President, Marketing"


#### Education

In [ ]:
len(degrees)

In [ ]:
degrees['degree_type'].dtype

In [ ]:
len(degrees['object_id'].unique())

We have degree details for the above number of people

We need to give a rating to the degrees. All Bachelors will have rating as 1. Masters will have rating 2 and PHD/MBA will have rating 3.As there are around 7000 differnet degrees, for other types of degrees we can use the median of the rest of the table. 

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('B\..*',na=False),'degree_score'] = 1

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('Bachelor',na=False),'degree_score'] = 1

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('^B.*',na=False),'degree_score'] = 1

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('JD',na=False),'degree_score'] = 1

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('Master',na=False),'degree_score'] = 2

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('^M.*',na=False),'degree_score'] = 2

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('M.$',na=False),'degree_score'] = 2

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('M\..*',na=False),'degree_score'] = 2

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('Ph\.*D',na=False),'degree_score'] = 3

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('MBA',na=False),'degree_score'] = 2

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('No degree',na=False),'degree_score'] = 0

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('LLB',na=False),'degree_score'] = 1

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('Foundation',na=False),'degree_score'] = 0.5

In [ ]:
degrees.loc[degrees['degree_type'] == '','degree_score'] = 1

In [ ]:
degrees['degree_score'].fillna(np.nanmedian(degrees['degree_score']), inplace=True)

In [ ]:
degrees[['degree_type','degree_score']]

In [ ]:
degrees.loc[degrees['degree_score'].isnull()]['degree_type'].describe()

In [399]:
len(degrees)

105215

In [398]:
degrees.drop_duplicates(subset=['object_id','degree_type'],inplace=True)

In [422]:
baseline_data = pd.merge(baseline_data, degrees, how='left', left_on=['object_id','degree_type'], right_on=['object_id','degree_type'])

In [423]:
baseline_data['degree_score'].fillna(np.nanmedian(baseline_data['degree_score']), inplace=True)

In [424]:
len(baseline_data1)

99327

In [406]:
len(baseline_data)

99327

In [ ]:
# education[['object_id','degree_score','degree_type']]
baseline_data[['degree_score','degree_type']]

In [387]:
education_with_no_persons = baseline_data[baseline_data['id_x'].isnull()]
person_with_no_education = baseline_data[baseline_data['id_y'].isnull()]
print "Education rows with no corresponding person",len(education_with_no_persons)
print "Persons with no details about their education",len(person_with_no_education)

Education rows with no corresponding person 92393
Persons with no details about their education 54265


In [386]:
education_with_no_persons = education[education['id_x'].isnull()]
person_with_no_education = education[education['id_y'].isnull()]
print "Education rows with no corresponding person",len(education_with_no_persons)
print "Persons with no details about their education",len(person_with_no_education)

Education rows with no corresponding person 0
Persons with no details about their education 0


Next Steps: 
1. We have sparse data for education. What to do with it?
2. How to clean workex data?

#### Funding Rounds

In [ ]:
len(funding_rounds)

In [ ]:
work_ex_funding = pd.merge(work_ex, funding_rounds,how='inner', left_on='relationship_object_id', right_on='object_id')

In [ ]:
len(work_ex_funding)

In [ ]:
work_ex_funding[:5]
funding_rounds['funded_at_datetime']= pd.to_datetime(funding_rounds['funded_at'])
funding_rounds[['funded_at_datetime','funded_at']]

In [ ]:
kevin = work_ex_funding[work_ex_funding['id_x']=='p:3']
kevin[['title','funded_at']]
kevin

In [ ]:
kevin1 = work_ex[work_ex['id_x']=='p:3']
# kevin1[['title','funded_at']]
# kevin1['start_at_datetime'] = pd.to_datetime(kevin['start_at'].dropna(),yearfirst = True)
# pd.to_datetime(kevin['end_at'])
# pd.to_datetime(kevin['end_at'],infer_datetime_format=True)
# kevin1[['start_at', 'end_at','start_at_datetime', 'end_at_datetime']]
# kevin1[['start_at', 'end_at']]
# pd.to_datetime(kevin1['start_at'].dropna())

In [ ]:
a = '2001-02-01'
pd.to_datetime(a)

In [ ]:
a = work_ex[:5].groupby('id_x')
work_ex[:10]

### Train Test and Evaluation Data

We need to divide the data into train test and evaluation data. We will divide it in the following ratio:

1.Train - 60%

2.Test - 20%

3.Evaluation - 20%


### Baseline Model

Our baseline model is dependent on Education and work experience of the founder. We will train our linear regression model on the score of the founder based on the above 2 features and run it on the test data. 

We will then find the error rate of our baseline model on the test data.

In [425]:
baseline_data['work_ex_score'] = baseline_data['funded_at'] - baseline_data['start_at']

In [426]:
baseline_data = pd.merge(companies, baseline_data, how='inner', left_on ='id', right_on='id_x')

In [427]:
negative_dates = baseline_data.loc[baseline_data['work_ex_score']<pd.Timedelta('0 days')]

In [428]:
len(negative_dates)

11284

In [429]:
baseline_data

,id,entity_type,entity_id,parent_id,name_x,normalized_name,permalink,category_code,status,founded_at,...,funding_round_code,funding_round_type,id_y,subject,institution,graduated_at,created_at_y,updated_at_y,degree_score,work_ex_score
0,c:1,Company,1,None,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,a,series-a,699.0,Electrical Engineering/Computer Science,"University of California, Berkeley",1994-01-01,2008-05-06 00:05:48,2008-05-06 00:05:48,1.0,11961 days
1,c:1,Company,1,None,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,a,series-a,41207.0,Economics (with honors),London School of Economics and Political Science,2000-01-01,2013-06-02 17:23:24,2013-06-02 17:23:24,1.0,-16 days
2,c:1,Company,1,None,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,a,series-a,41317.0,Business Administration,"University of California, Berkeley, Walter A. ...",1995-01-01,2013-06-03 10:54:04,2013-06-03 10:54:04,1.0,38259 days
3,c:1,Company,1,None,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,a,series-a,79998.0,Business Administration,University of Puget Sound,1980-01-01,2013-09-13 11:58:34,2013-09-13 11:58:34,1.0,-457 days
4,c:1,Company,1,None,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,b,series-b,699.0,Electrical Engineering/Computer Science,"University of California, Berkeley",1994-01-01,2008-05-06 00:05:48,2008-05-06 00:05:48,1.0,12418 days
5,c:1,Company,1,None,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,b,series-b,41207.0,Economics (with honors),London School of Economics and Political Science,2000-01-01,2013-06-02 17:23:24,2013-06-02 17:23:24,1.0,441 days
6,c:1,Company,1,None,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,b,series-b,41317.0,Business Administration,"University of California, Berkeley, Walter A. ...",1995-01-01,2013-06-03 10:54:04,2013-06-03 10:54:04,1.0,38716 days
7,c:1,Company,1,None,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,b,series-b,79998.0,Business Administration,University of Puget Sound,1980-01-01,2013-09-13 11:58:34,2013-09-13 11:58:34,1.0,0 days
8,c:1,Company,1,None,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,c,series-c+,699.0,Electrical Engineering/Computer Science,"University of California, Berkeley",1994-01-01,2008-05-06 00:05:48,2008-05-06 00:05:48,1.0,12922 days
9,c:1,Company,1,None,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,c,series-c+,41207.0,Economics (with honors),London School of Economics and Political Science,2000-01-01,2013-06-02 17:23:24,2013-06-02 17:23:24,1.0,945 days


In [430]:
for index,row in negative_dates.iterrows():
#     print row['funded_at'], companies.loc[companies['id']== row['id_x'],'founded_at']
#     print row['funded_at'] - companies.loc[companies['id']== row['id_x'],'founded_at']
#     print row['work_ex_score']
    if  row['funded_at'] and row['founded_at']:
        baseline_data.loc[index,'work_ex_score'] = row['funded_at'] - row['founded_at']
        row['work_ex_score']
#     baseline_data.loc[baseline_data['object_id']==row['object_id'],'work_ex_score'] = row['work_ex_score']
#     print baseline_data.loc[baseline_data['object_id']==row['object_id'],'work_ex_score']

In [431]:
len(baseline_data)

99327

In [432]:
 baseline_data.loc[baseline_data['work_ex_score']<pd.Timedelta('0 days'),'work_ex_score'] = 0

In [433]:
baseline_data.columns

Index([u'id', u'entity_type', u'entity_id', u'parent_id', u'name_x',
       u'normalized_name', u'permalink', u'category_code', u'status',
       u'founded_at', u'closed_at', u'domain', u'homepage_url',
       u'twitter_username', u'logo_url', u'logo_width', u'logo_height',
       u'short_description', u'description', u'overview', u'tag_list',
       u'country_code', u'state_code', u'city', u'region',
       u'first_investment_at', u'last_investment_at', u'investment_rounds',
       u'invested_companies', u'first_funding_at', u'last_funding_at',
       u'funding_rounds', u'funding_total_usd', u'first_milestone_at',
       u'last_milestone_at', u'milestones', u'relationships', u'created_by',
       u'created_at_x', u'updated_at_x', u'degree_type', u'object_id',
       u'start_at', u'title', u'first_name', u'last_name', u'name_y', u'id_x',
       u'funding_round_id', u'funded_at', u'funding_round_code',
       u'funding_round_type', u'id_y', u'subject', u'institution',
       u'graduated

In [ ]:
baseline_data.columns#.drop_duplicates(inplace=True)

In [435]:
final_data  = baseline_data[['name_x','work_ex_score','degree_score',u'funding_round_code',
               'first_name','last_name','funded_at']]#.groupby('funded_at').sum()

In [436]:
final_data.drop_duplicates(inplace=True)

In [440]:
final_data['work_ex_score'].fillna(0,inplace=True)

In [441]:
final_data['work_ex_score_i'] = (final_data['work_ex_score']/ np.timedelta64(1, 'D')).astype(int)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [442]:
final_data['work_ex_score_i'] = np.ceil(final_data['work_ex_score_i']/365)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [443]:
data = final_data.groupby(['name_x','funding_round_code'])[['degree_score','work_ex_score_i']].sum()

In [448]:
len(data)

32593

In [446]:
data = data.reset_index(1)

In [ ]:
len(final_data['name'].unique())

In [ ]:
len(baseline_data['name'].unique())

In [ ]:
len(baseline_data)

In [ ]:
data

In [449]:
def split_data(data):
    msk = np.random.rand(len(data)) < 0.8
    train_data = data[msk]
#     test = data[~msk]
#     msk = np.random.rand(len(test)) < 0.5
#     evaluation_data = test[msk]
    test_data = data[~msk]
    return train_data,test_data

In [450]:
data['funding_round_category'] = data['funding_round_code'].astype('category').cat.codes

In [451]:
from sklearn import svm

In [452]:

clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)

for i in xrange(5):
    train_data, test_data = split_data(data)
    clf.fit(train_data[['degree_score','work_ex_score_i','funding_round_category']])
#     y_pred_train = clf.predict(train_data[['degree_score','work_ex_score_i','funding_round_category']])
    y_pred_test = clf.predict(test_data[['degree_score','work_ex_score_i','funding_round_category']])
    print "Train data ", len(train_data)
    print "Test data ", len(test_data)
    print "Error ", float(len(y_pred_test[y_pred_test==-1]))/float(len(y_pred_test))

Train data  26091
Test data  6502
Error  0.145955090741
Train data  26076
Test data  6517
Error  0.144851925733
Train data  26198
Test data  6395
Error  0.13260359656
Train data  26112
Test data  6481
Error  0.130072519673
Train data  25996
Test data  6597
Error  0.124298923753


In [ ]:
len(y_pred_train[y_pred_train==-1])
len(y_pred_train)

In [ ]:
y_pred_test = clf.predict(test_data[['degree_score','work_ex_score_i','funding_round_category']])

In [ ]:
len(y_pred_test[y_pred_test==-1])
len(y_pred_test)

In [ ]:
y_pred_eval = clf.predict(evaluation_data[['degree_score','work_ex_score_i','funding_round_category']])

In [ ]:
len(y_pred_eval[y_pred_eval==-1])
len(y_pred_eval)

In [ ]:
len(data)

In [ ]:
0.146616541353 + 0.135338345865 + 0.158360128617+ 0.147034708937+0.148454157783

In [ ]:
0.735803882555/5

In [453]:
14.59+14.48+13.26+13+12.42

67.75

In [454]:
67.75/5

13.55

99327